In [2]:
import pandas as pd
import numpy as np

In [61]:
# read in data
customer_df = pd.read_csv("data/current_customers.csv")
# network_df = pd.read_csv("data/example_networks.csv")

/tmp/ipykernel_63422/2486453810.py:2: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  customer_df = pd.read_csv("data/current_customers.csv")


In [62]:
# get type of columns and column names
print(customer_df.dtypes)
print(customer_df.columns)

acct_id                        object
extenders                       int64
wireless_clients_count          int64
wired_clients_count             int64
rx_avg_bps                    float64
tx_avg_bps                    float64
rx_p95_bps                    float64
tx_p95_bps                    float64
rx_max_bps                    float64
tx_max_bps                    float64
rssi_mean                     float64
rssi_median                     int64
rssi_max                        int64
rssi_min                        int64
network_speed                  object
city                           object
state                          object
whole_home_wifi                object
wifi_security                  object
wifi_security_plus             object
premium_tech_pro               object
identity_protection            object
family_identity_protection     object
total_shield                   object
youtube_tv                     object
dtype: object
Index(['acct_id', 'extenders', 'wire

In [77]:
# calculate the averages of each column
# exclude the following columns ['acct_id', 'city', 'state']
# special case for 'network_speed' (convert to numeric first)
# if column is numeric, calculate the average
# otherwise count all values that are true and divide by the total number of rows

exclude_cols = ['acct_id', 'city', 'state']
avgs = {}
temp_df = customer_df.copy().drop(exclude_cols, axis=1)
for col in temp_df.columns:
    if col == 'network_speed':
        numeric_col = pd.to_numeric(temp_df[col].str.replace("M", "", regex=False), errors="coerce")
        avgs[col] = numeric_col.mean()
    elif pd.api.types.is_numeric_dtype(temp_df[col]):
        avgs[col] = temp_df[col].mean()
    else:
        pass

avgs_df = pd.DataFrame(avgs, index=[0])
print(avgs_df)
avgs_df.to_csv("data/avg_values.csv", index=False)

   extenders  wireless_clients_count  wired_clients_count    rx_avg_bps  \
0    0.72163               12.016019             4.111797  489285.28892   

      tx_avg_bps    rx_p95_bps     tx_p95_bps    rx_max_bps    tx_max_bps  \
0  298210.885731  1.444721e+06  398305.272225  3.228119e+06  862704.04209   

   rssi_mean  rssi_median  rssi_max   rssi_min  network_speed  
0 -57.786995   -58.599916 -33.71203 -80.987907     874.734559  


In [ ]:
from geopy.geocoders import Nominatim

def get_lat_lon(city, state):
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(f"{city}, {state}")
    if location is None:
        return None, None
    return location.latitude, location.longitude

In [78]:

# preprocess the data (get longitude, latitude, radius, and data)
grouped_df = customer_df.groupby(["city", "state"])
# create new df with latitude, longitude, radius, and color
new_df = pd.DataFrame(columns=["latitude", "longitude", 'extenders', 'wireless_clients_count', 'wired_clients_count', 'rx_avg_bps', 'tx_avg_bps', 'rx_p95_bps', 'tx_p95_bps', 'rx_max_bps','tx_max_bps', 'rssi_mean', 'rssi_median', 'rssi_max', 'rssi_min', 'network_speed'])
for (city, state), group in grouped_df:
    location_data = get_lat_lon(city, state)
    if (location_data[0] is None) or (location_data[1] is None):
        print(f"Could not find location data for {city}, {state}")
        continue
    avg_data = {}
    for col in group.columns:
        if col == 'network_speed':
            numeric_col = pd.to_numeric(group[col].str.replace("M", "", regex=False), errors="coerce")
            avg_data[col] = numeric_col/avgs_df[col].values[0]*500
        elif pd.api.types.is_numeric_dtype(group[col]):
            avg_data[col] = group[col].mean()/avgs_df[col].values[0]*500
        else:
            pass
    temp_df = pd.DataFrame(avg_data, index=[0])
    temp_df['latitude'] = location_data[0]
    temp_df['longitude'] = location_data[1]
    # Concatenate the temporary DataFrame with the new DataFrame
    new_df = pd.concat([new_df, temp_df], ignore_index=True)
new_df.head()
new_df.to_csv("data/preprocessed_customers.csv", index=False)

/tmp/ipykernel_63422/1346210460.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, temp_df], ignore_index=True)


Could not find location data for BERKELEY SPGS, WV
Could not find location data for BERKLEY SPRINGS, WV
Could not find location data for CONYNGHAM BORO, PA
Could not find location data for HARVEYS LAKE BORO, PA
Could not find location data for HUNTINTON, WV
Could not find location data for JPV, FL
Could not find location data for LE ROY, WV
Could not find location data for LKSID MARBLHD, OH
Could not find location data for LYKENS BORO, PA
Could not find location data for NCTY, FL
Could not find location data for NEW HOLLAND BOROUGH, PA
Could not find location data for NEW PHILA, OH
Could not find location data for NICHOLSON BORO, PA
Could not find location data for OLD RIVER-WINFRE, TX
Could not find location data for PALOS VERDES PENINSU, CA
Could not find location data for RIO GRANDE CY, TX
Could not find location data for SCTY, FL
Could not find location data for STAFFORD SPGS, CT
Could not find location data for VERNON ROCKVL, CT
Could not find location data for WILMINGTON-CNTY RTE

FileNotFoundError: [Errno 2] No such file or directory: 'data1.csv'